<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/HF_LLAMA2_LangChain_Moderation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">Learn by Doing LLM Projects</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Create a Moderation system with LLAMA-2 LangChain and HF.</h2>
    by <b>Pere Martra</b>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

This notebook needs an environment with GPU. I'm using a A100 GPU but it can run with any 16GB GPU.

# How To Create a Moderation System Using LangChain & Hugging Face.

We are going to create a Moderation System based in two Models. The first Model  reads the User comments and answer them.

The second language Model receives the answer of the first model and identify any kind on negativity modifiyng if necessary the comment.

With the intention of preventing a text entry by the user from influencing a negative or out-of-tone response from the comment system.

In [1]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.1
!pip install -q transformers==4.35.2
!pip install -q accelerate==0.26.1
!pip install -q xformers==0.0.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
pip show xformers

Name: xformers
Version: 0.0.23
Summary: XFormers: A collection of composable Transformer building blocks.
Home-page: https://facebookresearch.github.io/xformers/
Author: Facebook AI Research
Author-email: oncall+xformers@xmail.facebook.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [3]:
!pip install huggingface_hub==0.20.2

In [4]:
from getpass import getpass
hf_key = getpass("Hugging Face Key: ")

Hugging Face Key: ··········


In [5]:
!huggingface-cli login --token $hf_key

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.
* LLMChain: To create chains, where the prompts or the results can pass from one step to another inside the chain.

In [6]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.llms import HuggingFacePipeline
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [7]:
import torch
import os
import numpy as np
from torch import cuda, bfloat16

In [8]:
#In a MAC Silicon the device must be 'mps'
# device = torch.device('mps') #to use with MAC Silicon
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [9]:
device

'cuda:0'

##Load the Model .

In [10]:
#You can try with any llama model, but you will need more GPU and memory as you increase the size of the model.
model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "meta-llama/Llama-2-7b-hf"


In [11]:
# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_key
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    use_auth_token=hf_key
)
model.eval()
print(f"Model loaded on {device}")


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_aut_token=hf_key)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [77]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    #temperature=0.0,
    do_sample=False,
    top_p=0,
    #trust_remote_code=True,
    repetition_penalty=1.1,
    return_full_text=False,
    device_map='auto'
)

assistant_llm = HuggingFacePipeline(pipeline=pipe)

## Create the template for the first model called assistant.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [78]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
User:" {customer_request}"
Comment:
"""

In [79]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.

In [68]:
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False
)
#the output of the formatted prompt will pass directly to the LLM.

To execute the chain created it's necessary to call the .run method of the chain, and pass the variables necessaries.

In our case: customer_request and sentiment.

In [69]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [80]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """Your product is a piece of shit. I want my money back!"""

In [85]:
# Our assistatnt working in 'nice' mode.
assistant_response=create_dialog(customer_request, "nice")
print(assistant_response['assistant_response'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"I apologize for any inconvenience caused by our product. Can you please provide more details about your issue so we can assist you better? We value your feedback and would like to make things right."

In this example, the comment is generated in response to a negative comment from a user who is unhappy with the product they received. The comment acknowledges the user's frustration and offers assistance in resolving the issue. By using similar language and tone as the user, the comment is more likely to be well-received and lead to a resolution of the problem.


In [93]:
#Our assistant running in rude mode.
assistant_response = create_dialog(customer_request, "very rude")
print(assistant_response['assistant_response'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"I apologize for any inconvenience caused by our product. However, I must inform you that it is not possible to refund your money as it is against our company's policy. Our products are of high quality and have been thoroughly tested before being released to the market. If you have any further questions or concerns, please feel free to ask."


Both Answers are similar LLAMA-2 is a relly polite Model, and I can't force it to be impolite, rude, or give a bad answer. But for sure that is possible to obtain something similar with prompt hackings
.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.

In [94]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

###
Original comment: {comment_to_moderate}
###
Edited comment:"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template
)

In [95]:
moderator_llm = assistant_llm

In [96]:
#We build the chain for the moderator.
moderator_chain = LLMChain(
    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
)  # the output of the prompt will pass to the LLM.

In [97]:
# To run our chain we use the .run() command
moderator_says = moderator_chain.run({"comment_to_moderate": assistant_response['assistant_response']})

print(f"moderator_says: {moderator_says}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


moderator_says:  "Thank you for bringing this to my attention. I understand that you may be disappointed with our product, and I apologize for any inconvenience it has caused. However, I'm afraid it is not possible to offer a refund as it goes against our company's policies. Our products are designed with great care and attention to detail, and we stand behind their quality. If you have any other questions or concerns, please don't hesitate to ask."


This answer is more polite that the one produce by the  **"rude" assistant**.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.

In [98]:
#The optput of the first chain must coincide with one of the parameters of the second chain.
#The parameter is defined in the prompt_template.
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="comment_to_moderate",
    verbose=False,
)

#verbose True because we want to see the intermediate messages.
moderator_chain = LLMChain(
    llm=moderator_llm,
    prompt=moderator_prompt_template,
    verbose=True
)

**SequentialChain** is used to link different chains and parameters.

It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [99]:
from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
assistant_moderated_chain = SequentialChain(
    chains=[assistant_chain, moderator_chain],
    input_variables=["sentiment", "customer_request"],
    verbose=True,
)

Lets use our Moderating System!

In [100]:
# We can now run the chain.
assistant_moderated_chain.run({"sentiment": "very rude", "customer_request": customer_request})



> Entering new SequentialChain chain...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

###
Original comment: "I apologize for any inconvenience caused by our product. However, I must inform you that it is not possible to refund your money as it is against our company's policy. Our products are of high quality and have been thoroughly tested before being released to the market. If you have any further questions or concerns, please feel free to ask."
###
Edited comment:


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



> Finished chain.

> Finished chain.


' "Thank you for bringing this to my attention. I understand that you may be disappointed with our product, and I apologize for any inconvenience it has caused. However, I\'m afraid it is not possible to offer a refund as it goes against our company\'s policies. Our products are designed with great care and attention to detail, and we stand behind their quality. If you have any other questions or concerns, please don\'t hesitate to ask."'

## Conclusions
As You can see how the moderator changes the answer of our assistant. Both are polites, but the one produces by the moderator is more formal.